In [ ]:
from pathlib import Path
import numpy as np

from aicsimageio import AICSImage
from aicsimageio.readers.ome_tiff_reader import OmeTiffReader
from aicsimageio.writers import OmeTiffWriter
import tifffile as tif

import sys
src_path = str(Path.cwd().parent)
if src_path not in sys.path:
    sys.path.append(src_path)

import src.d00_utils.utilities as utils
import src.d00_utils.dirnames as dn

In [ ]:
seg_dir = Path(input())

In [ ]:
assert seg_dir.is_dir()
orig_dir = seg_dir.parent / dn.raw_ometif_dirname
assert orig_dir.is_dir()

In [ ]:
# Get channel subset info from segmentation directory
start_idx = np.char.find(seg_dir.name, 'chsubset')
chsubset_info = seg_dir.name[start_idx:]

save_dirname = f'{orig_dir.name}_{chsubset_info}_stacked'
save_dirpath = Path(orig_dir).parent / save_dirname
print(f'Full directory path: {save_dirpath}')

In [ ]:
# Stack single-class segmentation onto the original image
save_dirpath.mkdir(parents=True, exist_ok=True)
orig_names = [path.name for path in orig_dir.glob('*.tif')]
seg_names = [path.name for path in seg_dir.glob('*.tif')]

print(f'{len(seg_names)} segmented images found. {len(orig_names) - len(seg_names)} images missing segmentations.')

for name in seg_names:
    orig_name = name.split('_seg')[0] + '.ome.tif'
    orig_path = orig_dir / orig_name
    if orig_path.is_file():
    
        img_file = AICSImage(orig_path, reader=OmeTiffReader)
        img = img_file.data  
        
        seg = tif.imread(seg_dir / name)
        caax_pos = np.expand_dims((seg==1), (1,2))
        
        stacked_img = np.concatenate([img, caax_pos], axis=1)
    
        ome_metadata = utils.construct_ome_metadata(stacked_img, physical_pixel_sizes=img_file.physical_pixel_sizes)
        OmeTiffWriter.save(stacked_img, (save_dirpath / orig_name), ome_xml=ome_metadata)

In [ ]:
save_dirpath.mkdir(parents=True, exist_ok=True)
seg_names = [path.name for path in seg_dir.glob('*.tif')]

for name in seg_names:
    orig_name = name.split('_seg')[0] + '.ome.tif'
    orig_path = orig_dir / orig_name
    if orig_path.is_file():
    
        img_file = AICSImage(orig_path, reader=OmeTiffReader)
        img = img_file.data  
        
        seg = tif.imread(seg_dir / name)
        caax_pos = (seg==1)
        caax_neg = (seg==2)
        cell = (seg > 0)
        
        stacked_img = np.concatenate([img, np.expand_dims(caax_pos, (0, 1, 2)), np.expand_dims(cell, (0, 1, 2)), 
                                      np.expand_dims(caax_neg, (0, 1, 2))], axis=1)
    
        ome_metadata = utils.construct_ome_metadata(stacked_img, physical_pixel_sizes=img_file.physical_pixel_sizes)
        OmeTiffWriter.save(stacked_img, (save_dirpath / orig_name), ome_xml=ome_metadata)